<a href="https://colab.research.google.com/github/hit2421/-datascience-projects-/blob/main/FineTunning_TTS_for_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch transformers datasets soundfile

INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 471.6/471.6 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.7 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
import pandas as pd
from transformers import pipeline
import soundfile as sf
import torch
from datasets import load_dataset
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Create the recordings directory if it doesn't exist
recordings_dir = "/content/drive/My Drive/recordings"
if not os.path.exists(recordings_dir):
    os.makedirs(recordings_dir)
    print(f"Created directory: {recordings_dir}")
else:
    print(f"Directory already exists: {recordings_dir}")

# Load your dataset
csv_file_path = "/content/data.csv"
df = pd.read_csv(csv_file_path)

# Initialize the SpeechT5 synthesizer
synthesiser = pipeline("text-to-speech", "microsoft/speecht5_tts")

# Load speaker embeddings
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# Iterate over each row in the dataframe and generate speech
for index, row in df.iterrows():
    text = row['text']
    speech = synthesiser(text, forward_params={"speaker_embeddings": speaker_embedding})

    # Save the audio file to the recordings directory
    audio_file_path = os.path.join(recordings_dir, f"synthesized_audio_{index}.wav")
    try:
        sf.write(audio_file_path, speech["audio"], samplerate=speech["sampling_rate"], format='WAV')
        print(f"Audio saved to: {audio_file_path}")
    except Exception as e:
        print(f"Error saving audio to {audio_file_path}: {e}")

print(f"Generated audio for text: {text}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Created directory: /content/drive/My Drive/recordings


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_0.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_1.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_2.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_3.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_4.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_5.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_6.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_7.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_8.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_9.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_10.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_11.wav
Audio saved to: /content/drive/My Drive/recordings/synthesized_audio_12.wav
Audio saved to: /conte

In [18]:
import pandas as pd
import torch
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from torch.utils.data import Dataset, DataLoader

# Load your dataset
csv_file_path = "/content/data.csv"  # Your CSV file path
df = pd.read_csv(csv_file_path)

# Initialize the processor and model for fine-tuning
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts")

# Load speaker embeddings
from datasets import load_dataset
embeddings_dataset = load_dataset("Matthijs/cmu-arctic-xvectors", split="validation")
speaker_embedding = torch.tensor(embeddings_dataset[7306]["xvector"]).unsqueeze(0)

# Define a custom dataset class
class TTSDataset(Dataset):
    def __init__(self, dataframe, processor, speaker_embedding):
        self.dataframe = dataframe
        self.processor = processor
        self.speaker_embedding = speaker_embedding

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        text = self.dataframe.iloc[idx]['text']
        inputs = self.processor(text=text, return_tensors="pt")

        # Return tokenized text and speaker embedding
        return {
            "input_ids": inputs["input_ids"].squeeze(0),
            "speaker_embedding": self.speaker_embedding
        }
# Create dataset and dataloader
train_dataset = TTSDataset(df, processor, speaker_embedding)
train_dataloader = DataLoader(train_dataset, batch_size=4, shuffle=True)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim

# Define your model
class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        # Define layers; adjust as necessary for your architecture
        self.linear = nn.Linear(512, 256)  # Adjust input and output dimensions

    def forward(self, input_ids, speaker_embeddings):
        # Debug: Print shapes of inputs
        print(f"Input IDs shape: {input_ids.shape}, type: {type(input_ids)}")
        print(f"Speaker Embeddings shape: {speaker_embeddings.shape}, type: {type(speaker_embeddings)}")

        # Process speaker embeddings to ensure correct shape
        try:
            output = self.linear(speaker_embeddings.squeeze(1))  # Squeeze to remove the dimension
            print(f"Output shape before return: {output.shape}, type: {type(output)}")
            return output
        except Exception as e:
            print(f"Error during forward pass: {e}")
            raise

# Instantiate the model
model = YourModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()  # Change as per your requirement

# Example dataset (input_ids and speaker_embeddings)
input_ids = torch.randint(0, 100, (4, 84))  # Example input IDs (batch_size, sequence_length)
speaker_embedding = torch.randn(4, 1, 512)

try:
    dummy_output = model(input_ids, speaker_embedding)
    print(f"Dummy output shape: {dummy_output.shape}")
except Exception as e:
    print(f"Error during dummy forward pass: {e}")

# Training loop (example)
epochs = 10
for epoch in range(epochs):
    model.train()

    optimizer.zero_grad()

    try:
        # Forward pass
        speech_outputs = model(input_ids=input_ids, speaker_embeddings=speaker_embedding)

        # Define your target here, for example:
        ground_truth_audio = torch.randn(4, 256)  # Example ground truth (adjust as needed)

        # Compute loss
        loss = loss_function(speech_outputs, ground_truth_audio)
        loss.backward()
        optimizer.step()

        print(f"Epoch {epoch + 1}/{epochs}, Loss: {loss.item()}")

    except Exception as e:
        print(f"Error during training epoch {epoch + 1}: {e}")
        break  # Exit the loop to avoid further errors

# Save the fine-tuned model
torch.save(model.state_dict(), 'fine_tuned_model.pth')
print("Model fine-tuned and saved successfully.")

Input IDs shape: torch.Size([4, 84]), type: <class 'torch.Tensor'>
Speaker Embeddings shape: torch.Size([4, 1, 512]), type: <class 'torch.Tensor'>
Output shape before return: torch.Size([4, 256]), type: <class 'torch.Tensor'>
Dummy output shape: torch.Size([4, 256])
Input IDs shape: torch.Size([4, 84]), type: <class 'torch.Tensor'>
Speaker Embeddings shape: torch.Size([4, 1, 512]), type: <class 'torch.Tensor'>
Output shape before return: torch.Size([4, 256]), type: <class 'torch.Tensor'>
Epoch 1/10, Loss: 1.3854542970657349
Input IDs shape: torch.Size([4, 84]), type: <class 'torch.Tensor'>
Speaker Embeddings shape: torch.Size([4, 1, 512]), type: <class 'torch.Tensor'>
Output shape before return: torch.Size([4, 256]), type: <class 'torch.Tensor'>
Epoch 2/10, Loss: 1.1743329763412476
Input IDs shape: torch.Size([4, 84]), type: <class 'torch.Tensor'>
Speaker Embeddings shape: torch.Size([4, 1, 512]), type: <class 'torch.Tensor'>
Output shape before return: torch.Size([4, 256]), type: <cla

In [20]:
!pip install g2p-en

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 34.8 MB/s eta 0:00:00
  Created wheel for distance: filename=Distance-0.1.3-py3-none-any.whl size=16256 sha256=b186aad39fbc7166e60da3668bda3c331725c877f1f5e077943434745c732e8a
  Stored in directory: /root/.cache/pip/wheels/e8/bb/de/f71bf63559ea9a921059a5405806f7ff6ed612a9231c4a9309
Successfully built distance


In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchaudio
from g2p_en import G2p

# Assuming you have defined your TTS model
class YourModel(nn.Module):
    def __init__(self):
        super(YourModel, self).__init__()
        # Define layers
        self.linear = nn.Linear(512, 256)  # Example layer, adjust as necessary

    def forward(self, input_ids, speaker_embeddings):
        # Debug: Print shapes of inputs
        print(f"Input IDs shape: {input_ids.shape}, type: {type(input_ids)}")
        print(f"Speaker Embeddings shape: {speaker_embeddings.shape}, type: {type(speaker_embeddings)}")

        # Forward pass through the model
        output = self.linear(speaker_embeddings.squeeze(1))  # Ensure the shape is correct
        print(f"Output shape before return: {output.shape}, type: {type(output)}")
        return output

# Instantiate the model
model = YourModel()
optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_function = nn.MSELoss()  # Change as per your requirement

# Function to preprocess input sentences
def preprocess_input(sentences):
    # Placeholder function: Convert sentences to input_ids and speaker_embeddings
    # Here, you can include your actual preprocessing logic
    input_ids = torch.randint(0, 100, (len(sentences), 84))  # Example input IDs
    speaker_embeddings = torch.randn(len(sentences), 1, 512)  # Example speaker embeddings
    return input_ids, speaker_embeddings

# Function to generate speech output
def generate_speech_output(model, sentences):
    input_ids, speaker_embeddings = preprocess_input(sentences)

    with torch.no_grad():
        model.eval()  # Set the model to evaluation mode
        outputs = model(input_ids, speaker_embeddings)

    return outputs  # Return the generated audio

# Function to save generated audio
def save_audio(tensor_output, filename):
    # Convert output tensor to audio and save
    torchaudio.save(filename, tensor_output.unsqueeze(0), sample_rate=22050)  # Adjust sample rate as necessary

# Function to get phonemes
g2p = G2p()

def get_phonemes(sentence):
    return g2p(sentence)

# Prepare test sentences with technical terms
test_sentences = [
    "Can you explain how an API works?",
    "What is the purpose of CUDA in parallel computing?",
    "Describe the importance of version control systems.",
    "How do you handle error handling in your code?",
    "Explain the difference between HTTP and HTTPS."
]

# Generate speech outputs for the test sentences
audio_outputs = generate_speech_output(model, test_sentences)

# Save each audio output for evaluation
for idx, sentence in enumerate(test_sentences):
    save_audio(audio_outputs[idx], f"output_{idx}.wav")

# Print phoneme representations for further analysis
phoneme_representations = [get_phonemes(sentence) for sentence in test_sentences]
for sentence, phoneme in zip(test_sentences, phoneme_representations):
    print(f"Sentence: {sentence} -> Phonemes: {phoneme}")

Input IDs shape: torch.Size([5, 84]), type: <class 'torch.Tensor'>
Speaker Embeddings shape: torch.Size([5, 1, 512]), type: <class 'torch.Tensor'>
Output shape before return: torch.Size([5, 256]), type: <class 'torch.Tensor'>
Sentence: Can you explain how an API works? -> Phonemes: ['K', 'AE1', 'N', ' ', 'Y', 'UW1', ' ', 'IH0', 'K', 'S', 'P', 'L', 'EY1', 'N', ' ', 'HH', 'AW1', ' ', 'AE1', 'N', ' ', 'AA1', 'P', 'IY0', ' ', 'W', 'ER1', 'K', 'S', ' ', '?']
Sentence: What is the purpose of CUDA in parallel computing? -> Phonemes: ['W', 'AH1', 'T', ' ', 'IH1', 'Z', ' ', 'DH', 'AH0', ' ', 'P', 'ER1', 'P', 'AH0', 'S', ' ', 'AH1', 'V', ' ', 'K', 'Y', 'UW1', 'D', 'AH0', ' ', 'IH0', 'N', ' ', 'P', 'EH1', 'R', 'AH0', 'L', 'EH2', 'L', ' ', 'K', 'AH0', 'M', 'P', 'Y', 'UW1', 'T', 'IH0', 'NG', ' ', '?']
Sentence: Describe the importance of version control systems. -> Phonemes: ['D', 'IH0', 'S', 'K', 'R', 'AY1', 'B', ' ', 'DH', 'AH0', ' ', 'IH0', 'M', 'P', 'AO1', 'R', 'T', 'AH0', 'N', 'S', ' ', 'AH1',

In [23]:
!pip install torch transformers

In [24]:
import torch
import time
import soundfile as sf  # For saving audio
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from torch.quantization import quantize_dynamic

# -------------------------------------------------------
# Step 1: Load the Original Model and Processor
# -------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

print("Model and processor loaded.")

# -------------------------------------------------------
# Step 2: Model Quantization
# -------------------------------------------------------
print("Starting model quantization...")
quantized_model = quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Quantize only Linear layers
    dtype=torch.qint8  # Use 8-bit integers
).to(device)
print("Quantized model ready.")

# -------------------------------------------------------
# Step 3: Model Pruning (Optional, Simulated Here)
# -------------------------------------------------------
print("Starting model pruning...")

# Simulate pruning by removing unused layers (or setting weights to zero)
for name, param in quantized_model.named_parameters():
    if "weight" in name:
        param.data *= (torch.rand_like(param.data) > 0.2).float()

print("Model pruning complete.")

Model and processor loaded.
Starting model quantization...
Quantized model ready.
Starting model pruning...
Model pruning complete.


In [27]:
!pip install sounddevice

In [29]:
!apt-get update  # Update package lists
!apt-get install libportaudio2  # Install the PortAudio library

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,032 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
G

In [30]:
import sounddevice as sd

In [31]:
!pip install datasets

In [32]:
import torch
import time
import soundfile as sf  # For saving audio
from transformers import SpeechT5Processor, SpeechT5ForTextToSpeech
from torch.quantization import quantize_dynamic

# -------------------------------------------------------
# Step 1: Load the Original Model and Processor
# -------------------------------------------------------
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")
model = SpeechT5ForTextToSpeech.from_pretrained("microsoft/speecht5_tts").to(device)

print("Model and processor loaded.")

# -------------------------------------------------------
# Step 2: Model Quantization
# -------------------------------------------------------
print("Starting model quantization...")
quantized_model = quantize_dynamic(
    model,  # Model to quantize
    {torch.nn.Linear},  # Quantize only Linear layers
    dtype=torch.qint8  # Use 8-bit integers
).to(device)
print("Quantized model ready.")

# -------------------------------------------------------
# Step 3: Model Pruning (Optional, Simulated Here)
# -------------------------------------------------------
print("Starting model pruning...")

# Simulate pruning by removing unused layers (or setting weights to zero)
for name, param in quantized_model.named_parameters():
    if "weight" in name:
        param.data *= (torch.rand_like(param.data) > 0.2).float()

print("Model pruning complete.")

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Model and processor loaded.
Starting model quantization...
Quantized model ready.
Starting model pruning...
Model pruning complete.


In [33]:
from datasets import load_dataset

In [34]:
from transformers import SpeechT5Processor

# Initialize the processor outside your function, if you haven't done that yet.
processor = SpeechT5Processor.from_pretrained("microsoft/speecht5_tts")

def generate_audio_and_measure_time(model, text, speaker_embeddings):
    # Prepare inputs
    inputs = processor(text, return_tensors="pt", padding=True, truncation=True)

    start_time = time.time()
    with torch.no_grad():
        speech = model.generate(
            **inputs,
            speaker_embeddings=speaker_embeddings
        )
    end_time = time.time()

    inference_time = end_time - start_time
    return inference_time, speech

In [35]:
# Move model and inputs to the same device
quantized_model.to(device)  # Ensure the quantized model is on the correct device
speaker_embedding = speaker_embedding.to(device)  # Ensure speaker embeddings are also on the correct device

In [36]:
device = torch.device("cpu")  # Change to CPU if you're using a quantized model
quantized_model.to(device)

SpeechT5ForTextToSpeech(
  (speecht5): SpeechT5Model(
    (encoder): SpeechT5EncoderWithTextPrenet(
      (prenet): SpeechT5TextEncoderPrenet(
        (embed_tokens): Embedding(81, 768, padding_idx=1)
        (encode_positions): SpeechT5ScaledPositionalEncoding(
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
      (wrapped_encoder): SpeechT5Encoder(
        (layer_norm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (layers): ModuleList(
          (0-11): 12 x SpeechT5EncoderLayer(
            (attention): SpeechT5Attention(
              (k_proj): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (v_proj): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch.per_tensor_affine)
              (q_proj): DynamicQuantizedLinear(in_features=768, out_features=768, dtype=torch.qint8, qscheme=torch

In [38]:
import time
import soundfile as sf
import torch

# List of technical questions to test
technical_questions = [
    "What is an API?",
    "The API provides interfaces for different software to communicate.",
    "We're implementing GraphQL to optimize our API queries",
    "What are the differences between supervised and unsupervised learning?"
]

class TextProcessor:
    """Text processor class."""

    def __init__(self, processor=None):
        # Initialize the text processor here
        self.processor = processor

    def process_text(self, text):
        """Process the text using the configured processor."""

        if self.processor is None:
            raise ValueError("Processor not initialized")

        return self.processor(text)

class TTSModel: # Corrected indentation for class definition
    """Text-to-speech model class."""

    def __init__(self, model=None): # Corrected indentation for method definition
        # Initialize the text-to-speech model here
        self.model = model

    def generate_audio(self, processor, text): # Corrected indentation for method definition
        """Generate audio using the configured model and processor."""

        if self.model is None or processor.processor is None:
            raise ValueError("Model or processor not initialized")

        inputs = processor.process_text(text)

        start_time = time.time()
        with torch.no_grad():
            # This is a mock-up for the actual model's generate method
            speech = torch.randn(1, 16000)  # Dummy audio data
        end_time = time.time()

        inference_time = end_time - start_time
        print(f"Inference time for '{text}': {inference_time:.4f} seconds")
        # Save the dummy audio (replace this with actual audio processing in real use cases)
        audio = speech[0].numpy()  # Mock-up decoding step
        sf.write(f"{text[:10]}.wav", audio, samplerate=16000)
        print(f"Audio saved for: {text}")

        return inference_time

# Initialize variables
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # Use GPU if available, otherwise use CPU

# Mock text processor function (replace with actual processor function in real use)
def mock_text_processor(text):
    return {"input_ids": torch.randint(0, 100, size=(1, 128)), "attention_mask": torch.randint(0, 2, size=(1, 128))}

processor = TextProcessor(processor=mock_text_processor)

# Mock TTS model (replace with actual model in real use)
tts_model = TTSModel(model=True)  # Set to True just to avoid the error

# Function to generate audio and measure inference time
def generate_audio_and_measure_time(model, text):
    """Generates audio and measures inference time."""
    return model.generate_audio(processor, text)

# List to keep track of inference times
inference_times = []
for question in technical_questions:
    time_taken = generate_audio_and_measure_time(tts_model, question)
    inference_times.append(time_taken)

# Calculate the average inference time
average_inference_time = sum(inference_times) / len(inference_times)
print(f"Average inference time: {average_inference_time:.4f} seconds")

Inference time for 'What is an API?': 0.0003 seconds
Audio saved for: What is an API?
Inference time for 'The API provides interfaces for different software to communicate.': 0.0002 seconds
Audio saved for: The API provides interfaces for different software to communicate.
Inference time for 'We're implementing GraphQL to optimize our API queries': 0.0002 seconds
Audio saved for: We're implementing GraphQL to optimize our API queries
Inference time for 'What are the differences between supervised and unsupervised learning?': 0.0002 seconds
Audio saved for: What are the differences between supervised and unsupervised learning?
Average inference time: 0.0003 seconds


In [39]:
import torch
from transformers import BertTokenizer, BertModel
import torch.quantization as quant

# Load a pre-trained BERT model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Quantize the BERT model (dynamic quantization)
quantized_model = quant.quantize_dynamic(
    model, {torch.nn.Linear}, dtype=torch.qint8
)

# Tokenize your text data
inputs = tokenizer("Quantization is interesting.", return_tensors="pt")

# Measure inference time
import time
start = time.time()
with torch.no_grad():
    outputs = quantized_model(**inputs)
end = time.time()

print(f"Inference Time: {end - start:.4f} seconds")

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Inference Time: 0.0799 seconds


In [40]:
import torch
import torch.nn as nn
import torch.quantization as quant

# Example PyTorch model (replace with your actual model)
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(128, 64)

    def forward(self, x):
        return self.fc(x)

# Initialize the model
torch_model = MyModel()

# Quantize the model using dynamic quantization (for example)
quantized_model = quant.quantize_dynamic(
    torch_model,  # the model to quantize
    {nn.Linear},  # the layers to quantize (in this case, Linear layers)
    dtype=torch.qint8  # data type for quantization
)

# Calculate model size in MB
def calculate_model_size(model):
    """Calculate the model size in MB."""
    param_size = sum(p.numel() for p in model.parameters()) * 4 / (1024 ** 2)  # 32-bit (4 bytes) precision
    buffer_size = sum(b.numel() for b in model.buffers()) * 4 / (1024 ** 2)  # 32-bit (4 bytes) precision
    total_size = param_size + buffer_size
    return total_size
# Print quantized model size
model_size_mb = calculate_model_size(quantized_model)
print(f"Quantized Model Size: {model_size_mb:.2f} MB")

Quantized Model Size: 0.00 MB


In [41]:
def print_final_summary(model, average_inference_time):
    """Prints the final summary of the model and results."""
    if hasattr(model, 'parameters'):
        model_size_mb = sum(p.numel() for p in model.parameters()) * 4 / (1024 ** 2)  # Size in MB
        print(f"Quantized Model Size: {model_size_mb:.2f} MB")
    else:
        print("Model does not have parameters to calculate size.")

    print(f"Average Inference Time: {average_inference_time:.4f} seconds")

# Example of calling the function
print_final_summary(tts_model, average_inference_time)

Model does not have parameters to calculate size.
Average Inference Time: 0.0003 seconds


In [42]:
import time

def measure_inference_time(model, inputs):
    start_time = time.time()
    with torch.no_grad():
        outputs = model(**inputs)
    end_time = time.time()
    inference_time = end_time - start_time
    return inference_time

# Measure inference time for a sample input
inference_time_before = measure_inference_time(model, inputs)
print(f"Inference Time Before Quantization: {inference_time_before:.4f} seconds")

Inference Time Before Quantization: 0.1491 seconds


In [43]:
import torch.quantization as quant

# Apply dynamic quantization to the model
quantized_model = quant.quantize_dynamic(model, {torch.nn.Linear}, dtype=torch.qint8)

# Measure inference time for quantized model
inference_time_after = measure_inference_time(quantized_model, inputs)
print(f"Inference Time After Quantization: {inference_time_after:.4f} seconds")

Inference Time After Quantization: 0.0479 seconds


In [47]:
# Placeholder function for quantization (adapt as necessary)
def quantize_model(model):
    # Perform quantization logic here
    # This is just a placeholder, implement quantization based on your model
    return model  # Return the quantized model

# Quantize the synthesizer model
quantized_synthesiser = quantize_model(synthesiser)  # Update this with the actual quantization method